<a href="https://colab.research.google.com/github/AidarZhumekenov/Deep-Learning/blob/main/DL_final_project_(Chess).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib as matplot
import seaborn as sns    
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressio
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, cross_val_predict
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score, classification_report, make_scorer, accuracy_score
import warnings
import time

warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [92]:
df = pd.read_csv('/content/drive/My Drive/Master KBTU/Deep Learning/Final/games final.csv', sep=';', decimal='.')
df.shape

(20058, 14)

In [97]:
df.head()

,id,rated,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [98]:
df.describe()

,turns,white_rating,black_rating,opening_ply
count,20058.000000,20058.000000,20058.000000,20058.000000
mean,60.465999,1596.631868,1588.831987,4.816981
std,33.570585,291.253376,291.036126,2.797152
min,1.000000,784.000000,789.000000,1.000000
25%,37.000000,1398.000000,1391.000000,3.000000
50%,55.000000,1567.000000,1562.000000,4.000000
75%,79.000000,1793.000000,1784.000000,6.000000
max,349.000000,2700.000000,2723.000000,28.000000


In [99]:
df_demo = pd.read_csv('/content/drive/My Drive/Master KBTU/Deep Learning/Final/games demo.csv', sep=';', decimal='.')
df_demo.shape

(3, 13)

In [100]:
df_demo.head()

,id,rated,turns,victory_status,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,AZH1,False,27,mate,15+15,FuocoNero,1253,jamboger,1267,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,e4,King's Pawn,1
1,AZH2,False,18,resign,15+15,jamboger,1267,FuocoNero,1253,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,d4,Queen's Pawn Game,1
2,AZH3,True,50,draw,45+45,jamboger,1258,nadgob,1620,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,c4,English Opening,1


In [8]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )


In [102]:
feature_columns = ['rated','turns','white_rating','black_rating','victory_status','increment_code','white_id','black_id','opening_eco']
target_column = 'winner'

def convert(data):
    number = preprocessing.LabelEncoder()
    data['rated'] = number.fit_transform(data.rated)
    data['white_id'] = number.fit_transform(data.white_id)
    data['black_id'] = number.fit_transform(data.black_id)
    data['increment_code'] = number.fit_transform(data.increment_code)
    data['victory_status'] = number.fit_transform(data.victory_status)
    data['opening_eco'] = number.fit_transform(data.opening_eco)
    data=data.fillna(-999)
    return data

df=convert(df)
df_demo=convert(df_demo)

In [48]:
df.head()

,id,rated,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,13,2,white,110,1287,1500,11,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,248,1387,4
1,l1NXvwaE,True,16,3,black,298,16,1322,7748,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,71,750,3
2,mIICvQHh,True,61,1,white,298,3823,1496,11,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,171,650,2
3,kWKvrqYL,True,61,1,white,187,2017,1439,134,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,241,992,2
4,9tXo1AUZ,True,95,1,white,247,6056,1523,134,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,192,783,4


In [18]:
# Correlation Matrix
def create_dummies(df,column_name):
  dummies = pd.get_dummies(df[column_name],prefix=column_name)
  df = pd.concat([df,dummies],axis=1)
  return df

df_corr = create_dummies(df, 'winner')
corr = df_corr.corr()
corr = (corr)
corr

,rated,created_at,last_move_at,turns,victory_status,increment_code,white_id,white_rating,black_id,black_rating,opening_eco,opening_name,opening_ply,winner_black,winner_draw,winner_white
rated,1.000000,-0.001171,-0.001176,0.090698,0.004556,-0.080649,-0.016895,0.003284,-0.013287,0.032655,0.018685,-0.004455,-0.001888,0.012417,-0.027358,-0.000743
created_at,-0.001171,1.000000,1.000000,0.083185,0.012867,-0.208100,-0.021912,0.116170,-0.000055,0.133316,0.042691,0.032298,0.100390,0.013830,0.024690,-0.024260
last_move_at,-0.001176,1.000000,1.000000,0.083196,0.012865,-0.208097,-0.021910,0.116172,-0.000054,0.133318,0.042690,0.032299,0.100392,0.013829,0.024694,-0.024261
turns,0.090698,0.083185,0.083196,1.000000,-0.214608,-0.019432,-0.014048,0.129753,-0.010745,0.160467,0.029353,0.015155,0.056348,0.007019,0.170648,-0.079486
victory_status,0.004556,0.012867,0.012865,-0.214608,1.000000,0.004379,0.018748,0.110852,0.012094,0.109694,0.021514,0.012020,0.063779,0.094297,-0.452193,0.098207
increment_code,-0.080649,-0.208100,-0.208097,-0.019432,0.004379,1.000000,0.037917,0.002457,0.041067,-0.022655,-0.053937,-0.021005,-0.047482,-0.011707,0.012912,0.006172
white_id,-0.016895,-0.021912,-0.021910,-0.014048,0.018748,0.037917,1.000000,0.020922,0.007068,0.011820,0.007950,0.010742,0.013674,0.000323,-0.026778,0.011054
white_rating,0.003284,0.116170,0.116172,0.129753,0.110852,0.002457,0.020922,1.000000,0.001637,0.634171,0.054661,0.022640,0.277877,-0.148371,0.045114,0.128577
black_id,-0.013287,-0.000055,-0.000054,-0.010745,0.012094,0.041067,0.007068,0.001637,1.000000,0.014674,0.006317,0.005404,0.015415,0.006514,-0.011107,-0.001768
black_rating,0.032655,0.133316,0.133318,0.160467,0.109694,-0.022655,0.011820,0.634171,0.014674,1.000000,0.026664,0.041206,0.255582,0.154778,0.040130,-0.171172


In [19]:
# Create a function to select features
# Note that feature names are stored in cols

def select_features(df):   
  all_X = df[feature_columns]
  all_y = df[target_column]

  clf = RandomForestClassifier(random_state=1, max_depth=5, class_weight="balanced", min_weight_fraction_leaf=0.01)
  selector = RFECV(clf)
  selector.fit(all_X, all_y)
  best_columns = list(all_X.columns[selector.support_])
  print('Best Columns \n' + '-'*12 + '\n' + '{}'.format(best_columns))

  return best_columns

In [20]:
best_cols = select_features(df)

Best Columns 
------------
['turns', 'white_rating', 'black_rating', 'victory_status']


In [ ]:
X = df[feature_columns]
y=df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1, stratify=y)

dtree = RandomForestClassifier(
    max_depth=5,
    random_state = 1,
    class_weight="balanced",
    min_weight_fraction_leaf=0.01
    )
dtree = dtree.fit(X_train,y_train)


## plot the importances ##
importances = dtree.feature_importances_
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(12,6))
plt.title("Feature importances by DecisionTreeClassifier")
plt.bar(range(len(indices)), importances[indices], color='lightblue',  align="center")
plt.step(range(len(indices)), np.cumsum(importances[indices]), where='mid', label='Cumulative')
plt.xticks(range(len(indices)), feature_columns[indices], rotation='vertical',fontsize=14)
plt.xlim([-1, len(indices)])
plt.show()

In [72]:
def select_model(df, features):
    
    all_X = df[features]
    all_y = df[target_column]


    #create a list of dics which contains models and hyperparameters
    models = [
        
        {
            "name": "Logistic Regression",
            "estimator": LogisticRegression(),
            "hyperparameters":
            {
            "penalty" : ['l2'],
            "C" : [1, 4, 20],
            "solver": ["newton-cg", "lbfgs"]   
            }
            
        },

        {
            "name": "XGBoost",
            "estimator": XGBClassifier(seed=42),
            "hyperparameters":
                {
                    "n_estimators": [10, 20, 50],
                    "objective" : ["multi:softmax"],
                    "learning_rate": [0.05, 0.1],
                    "max_depth": [3, 5, 10],
                    "min_child_weight" : [3,5],
                    "warm_start" : ["true"],
                    "subsample" : [0.8],
                    "gamma" : [0.4],
                    "colsample_bytree" : [0.8],
                    "scale_pos_weight" : [1],
                    "reg_alpha":[1e-5]                
                 }
        },

        {
            "name": "RandomForestClassifier",
            "estimator": RandomForestClassifier(random_state=1),
            "hyperparameters":
                {
                    "n_estimators": [10, 20],
                    "criterion": ["entropy", "gini"],
                    "max_depth": [5, 12],
                    "max_features": ["log2", "sqrt"],
                    "min_samples_leaf": [1, 5, 8],
                    "min_samples_split": [2, 3, 5]

                }
        },
                {
            "name": "Multi Layer Perceptron (MLP)",
            "estimator": MLPClassifier(random_state=1),
            "hyperparameters":
                {
                    "hidden_layer_sizes": [(10,10), (20,20)],
                    "activation": ["relu", "tanh", "logistic"],
                    "solver": ['sgd', 'adam','lbfgs'],
                    "alpha" : [0.001, 0.05],
                    "max_iter" : [100],
                    "learning_rate": ["constant", "adaptive"],
                    "warm_start" : ["true"]

                }
        }
        
    ]
    
    for model in log_progress(models):
        print(model["name"])
        print("-"*len(model["name"]))
        
        grid = GridSearchCV(model["estimator"],
                            param_grid=model["hyperparameters"],
                            scoring = 'accuracy',
                            cv=3)

        grid.fit(all_X,all_y)
        model["best_params"] = grid.best_params_
        model["best_score"] = grid.best_score_
        model["best_model"] = grid.best_estimator_
        model["scoring"] = grid.scorer_
        
        print("Best Paramerters:\n" + "{}".format(model["best_params"]))
        print("Best Score:\n" + "{}".format(model["best_score"]))
        print("Best Model:\n" + "{}\n".format(model["best_model"]))
        print("Scoring method:\n" + "{}\n".format(model["scoring"]))
        
    return models


In [73]:
models = select_model(df, best_cols)

Logistic Regression
-------------------
Best Paramerters:
{'C': 4, 'penalty': 'l2', 'solver': 'newton-cg'}
Best Score:
0.6671153654402233
Best Model:
LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

Scoring method:
make_scorer(accuracy_score)

XGBoost
-------
Best Paramerters:
{'colsample_bytree': 0.8, 'gamma': 0.4, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 3, 'n_estimators': 50, 'objective': 'multi:softmax', 'reg_alpha': 1e-05, 'scale_pos_weight': 1, 'subsample': 0.8, 'warm_start': 'true'}
Best Score:
0.7825805165021439
Best Model:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4,
              learning_rate=

In [87]:
X_train, X_test, y_train, y_test = train_test_split(df[feature_columns], df[target_column], test_size=0.1, random_state=42)

xgb_model = xgb.XGBClassifier(objective="multi:softmax", random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [88]:
y_pred

array(['white', 'white', 'white', ..., 'draw', 'white', 'white'],
      dtype=object)

In [103]:
# DEMO with XGBoost softmax
# Split the data into training/testing sets
X_train = df[feature_columns]
X_test = df_demo[feature_columns]
y_train = df[target_column]

xgb_model = xgb.XGBClassifier(objective="multi:softmax", random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_test = xgb_model.predict(X_test)

In [104]:
y_pred_test

array(['black', 'black', 'draw'], dtype=object)

In [107]:
# DEMO with LogReg lbgfs
# Split the data into training/testing sets

regr = linear_model.LogisticRegression(solver='lbfgs',multi_class='multinomial')
regr.fit(X_train, y_train)
y_pred_test2 = regr.predict(X_test)

In [108]:
y_pred_test2

array(['white', 'white', 'black'], dtype=object)